In [1]:
import json
import os
from collections import defaultdict
from tqdm import tqdm


In [2]:

DATA_DIR= './output_fold'
file_path= [os.path.join(DATA_DIR, path) for path in os.listdir(DATA_DIR)]

vote_dict= defaultdict(dict)
org_data= []
for idx, file in tqdm(enumerate(file_path)):
  with open(file, 'r') as f:
    for i, line in enumerate(f):
      line= json.loads(line)
      id, sentence, annotation= line['id'], line['sentence_form'], line['annotation']

      if idx==0:
        vote_dict[id]['aspect']= defaultdict(lambda: 0)
        vote_dict[id]['sentiment']= defaultdict(lambda: 0)
        org_data.append(line)
      
      vote_dict[id]['sentence_form']= sentence
      
      for annot in annotation:
        vote_dict[id]['aspect'][annot[0]]+=1
        vote_dict[id]['sentiment'][annot[1]]+=1
  

5it [00:00, 56.07it/s]


In [3]:
print(len(vote_dict.keys()))

2127


In [4]:
keys= list(vote_dict.keys())
vote_dict[keys[0]]

ASPECT_THRESHOLD= 2
SENTIMENT_THRESHOLD= 3

submission= []

annot_dict= defaultdict(lambda : 0)
with open('./data/nikluge-sa-2022-test.jsonl', 'r') as f:
  for i, line in enumerate(f):
    line= json.loads(line)
    id, sentence, annotation= line['id'], line['sentence_form'], line['annotation']

    example= vote_dict[id]
    aspect_candidate= []
    sentiment_candidate= []
    use_sentiment= ''

    annot= []

    max_aspect_value= 0
    for k, v in (example['aspect'].items()):
      if max_aspect_value < v:
        max_aspect_value= v
      
    max_sentiment_value= 0
    for k, v in example['sentiment'].items():
      if max_sentiment_value < v:
        max_sentiment_value= v
      
    # print(max_aspect_value, ASPECT_THRESHOLD)
    for k, v in example['aspect'].items():
      if v >= min(max_aspect_value, ASPECT_THRESHOLD):
        aspect_candidate.append((k, v))
    
    for k, v in (example['sentiment'].items()):
      if v >= min(max_sentiment_value, SENTIMENT_THRESHOLD):
        sentiment_candidate.append((k, v))
      
      if v == max_sentiment_value:
        use_sentiment= k
        break;
    
    for k, v in aspect_candidate:
      annot.append([k, use_sentiment])
    
    annot_dict[len(annot)]+=1

    submission.append({'id': id, 'sentence_form': sentence, 'annotation': annot})

outputfile_name= './submission_final.jsonl'
with open(outputfile_name , encoding= "utf-8" ,mode="w") as file: 
    for i in submission: 
        file.write(json.dumps(i,ensure_ascii=False) + "\n")
  
